<a href="https://colab.research.google.com/github/AchrafAsh/gnn-receptive-fields/blob/main/sparse_graph_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
import os.path as osp
from google.colab import drive, files
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
try:
    os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
except:
    pass
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [2]:
import time
import concurrent.futures
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch_sparse import spspmm
from torch_sparse.tensor import SparseTensor
from collections import Counter
from torch_geometric.utils import to_dense_adj
from tqdm.notebook import tqdm

%matplotlib inline
sns.set_style('whitegrid')

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [3]:
%%capture
!wget https://raw.githubusercontent.com/AchrafAsh/gnn-receptive-fields/main/data.py

from data import load_dataset
path = osp.join(os.getcwd(), 'data')

In [4]:
cora_dataset = load_dataset(path, 'Cora')
G = cora_dataset[0]

Processing...
Done!


In [5]:
sparse_edge_index = torch.sparse_coo_tensor(G.edge_index, torch.ones(G.edge_index.size(1)), (G.num_nodes, G.num_nodes))

In [6]:
sec_edge_index = torch.sparse.mm(sparse_edge_index, sparse_edge_index).coalesce()

In [70]:
def clamp(x: torch.Tensor):
    if not x.is_coalesced(): x = x.coalesce()

    mask = (x._values() > 0).nonzero().view(-1)
    values = x._values().index_select(0, mask).clamp(0, 1)
    indices = x._indices().index_select(1, mask)

    return torch.sparse_coo_tensor(indices, values, x.shape)

In [58]:
x = (sec_edge_index - sparse_edge_index).coalesce()

In [59]:
x = clamp(x)

tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [60]:
x + sparse_edge_index

tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [   0,  633,  926,  ..., 2138, 2706, 2707]]),
       values=tensor([1., 1., 1.,  ..., 1., 2., 1.]),
       size=(2708, 2708), nnz=99596, layout=torch.sparse_coo)

In [54]:
x.coalesce()

tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [   0,  633,  926,  ..., 2138, 2706, 2707]]),
       values=tensor([3., 0., 1.,  ..., 1., 2., 4.]),
       size=(2708, 2708), nnz=99596, layout=torch.sparse_coo)

In [77]:
def sparse_hop_neighbors(k:int, edge_index: torch.Tensor, num_nodes:int):
    # transform edge_index into a sparse tensor
    sparse_edge_index = torch.sparse_coo_tensor(edge_index, torch.ones(edge_index.size(1)), (num_nodes, num_nodes))

    cum_neighbors = neighbors = pow_A = sparse_edge_index.clone()

    yield neighbors, cum_neighbors

    for _ in range(1, k):
        pow_A = clamp(torch.sparse.mm(sparse_edge_index, pow_A))
        neighbors = clamp(pow_A - cum_neighbors)
        cum_neighbors = cum_neighbors + neighbors
        
        yield neighbors, cum_neighbors

In [84]:
def tensor_memory(x:torch.Tensor):
    return x.element_size() * x.nelement() / 1e9

In [86]:
def dense_hop_neighbors(k:int, edge_index: torch.Tensor):
    dense_adj = to_dense_adj(edge_index).squeeze(0)
    cum_neighbors = pow_A = dense_adj.clone()

    yield dense_adj, dense_adj

    for _ in range(k-1):
        pow_A = torch.mm(dense_adj, pow_A)
        neighbors = torch.where(
            (torch.clamp(pow_A, max=1, min=0) - cum_neighbors) > 0,
            1, 0)
        cum_neighbors += neighbors
        
        yield neighbors, cum_neighbors

In [87]:
start = time.time()
for neighbors, cum_neighbors in tqdm(sparse_hop_neighbors(10, G.edge_index, G.num_nodes), total=10):
    print(tensor_memory(cum_neighbors))
end = time.time()
print(end - start)

  0%|          | 0/20 [00:00<?, ?it/s]

0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
10.680636405944824


In [88]:
start = time.time()
for neighbors, cum_neighbors in tqdm(dense_hop_neighbors(10, G.edge_index), total=10):
    print(tensor_memory(cum_neighbors))
end = time.time()
print(end - start)

  0%|          | 0/20 [00:00<?, ?it/s]

0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
0.029333056
6.058846950531006
